# Curso Kaggle - Feature Engineering

https://www.kaggle.com/learn/feature-engineering

In [1]:
import pandas as pd
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

O objetivo do Feature Engineering é simplesmente tornar seus dados mais adequados ao problema em questão.

* Melhorar o desempenho preditivo de um modelo
* Reduzir as necessidades computacionais ou de dados
* Melhorar a interpretabilidade dos resultados

Princípio orientador da engenharia de recursos.

Para que um recurso seja útil, ele deve ter um relacionamento com o destino que seu modelo seja capaz de aprender.

Os modelos lineares, por exemplo, só são capazes de aprender relações lineares. 

Portanto, ao usar um modelo linear, seu objetivo é transformar os recursos para tornar linear seu relacionamento com o alvo.

## Exemplo

Para ilustrar essas ideias, veremos como adicionar alguns recursos sintéticos a um conjunto de dados pode melhorar o desempenho preditivo de um modelo de floresta aleatória.

O conjunto de dados Concrete contém uma variedade de formulações de concreto e a resistência à compressão do produto resultante, que é uma medida de quanta carga esse tipo de concreto pode suportar. A tarefa para este conjunto de dados é prever a resistência à compressão de um concreto dada a sua formulação.

In [3]:
concrete = pd.read_excel('/home/franciscofoz/Documents/GitHub/machine-learning-training/Datasets/Concrete_Data.xls')
concrete

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.284354
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.178794
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.696601
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.768036


Você pode ver aqui os vários ingredientes que entram em cada variedade de concreto. Veremos em breve como adicionar alguns recursos sintéticos adicionais derivados deles pode ajudar um modelo a aprender relacionamentos importantes entre eles.

Primeiro, estabeleceremos uma linha de base treinando o modelo no conjunto de dados não aumentado. Isso nos ajudará a determinar se nossos novos recursos são realmente úteis.

Estabelecer linhas de base como essa é uma boa prática no início do processo de engenharia de recursos. Uma pontuação de linha de base pode ajudá-lo a decidir se vale a pena manter seus novos recursos ou se você deve descartá-los e possivelmente tentar outra coisa.